In [1]:
!sh directorio.sh

El directorio ya existe warehouse/raw
La base de datos '03042024.parquet' existe en 'base.data'
Copiando la base de datos a 'warehouse/raw'...
La base de datos se ha copiado exitosamente a 'warehouse/raw'


In [10]:
!sh dir_hdfs.sh

Creando directorio /user/warehouse...
Creando directorio /user/warehouse/bronze...
Creando directorio /user/warehouse/silver...
Creando directorio /user/warehouse/gold...


In [ ]:
!pip3 install shapely
!pip3 install geopandas


In [27]:
# Crear el dataset crudo

!python3 01_gen_bd/create_data.py

In [27]:
# Crear el dataset crudo

!python3 01_gen_bd/create_data.py

In [ ]:
def load_parquet():
        dire = "base.data"
        base = "16232203042024.parquet"
        df_cust = pd.read_parquet(os.path.join(dire, "\\",base))

def silver(df_cust):
        df_cust[['latitude', 'longitude']] = df_cust['points'].apply(lambda geom: pd.Series(obtener_coordenadas(geom)))    
        df_cust['event_date'] = df_cust['fecha'].astype(str)
        df_cust[['event_date', 'event_hour']] = df_cust['event_date'].str.split(' ', expand=True)
        df_cust[['event_year', 'event_month', 'event_day']] = df_cust['event_date'].str.split('-', expand=True)
        df_cust[['event_hour', 'event_minute', 'event_second']] = df_cust['event_hour'].str.split(':', expand=True)
        df_cust['partition_date'] = df_cust['event_day'] +  df_cust['event_month'] + df_cust['event_year']
        df_cust = df_cust.rename(columns={'NOMBRE': 'neighborhood'})
        df_cust = df_cust.rename(columns={'IDENTIFICACION': 'commune'})
        df_cust = df_cust.rename(columns={'empl_employee_id': 'employee_id'})    
        generar_order_id(df_cust)

        cols =['partition_date', 'order_id', 'commune', 'customer_id', 'employee_id', 
                'event_date', 'event_day', 'event_hour', 'event_minute', 
                'event_month', 'event_second', 'event_year', 'latitude', 'longitude',
                'neighborhood', 'quantity_products']
        df_cust = df_cust[cols]

        file_name = df_cust["partition_date"][0]

        df_cust.to_parquet(file_name)

In [ ]:
# Crea el dataset en silver
!python3 01_gen_bd/silver.py

In [ ]:
!hdfs dfs 

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("trabajo_final").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
from pyspark import SparkFiles

# Preparar el archivo en spark para cargar
file: str = "warehouse/raw/03042024.parquet"
spark.sparkContext.addFile(file)

spark_file: str = "file://"+SparkFiles.get("03042024.parquet")
print(spark_file)

file:///tmp/spark-3d415965-f786-427a-9c09-dbb7c2100b0b/userFiles-dcdc2bf2-0d7d-418f-949b-fe4747bacf9c/03042024.parquet


In [5]:
df_comunas = spark.read.parquet(spark_file)

In [7]:
df_comunas.describe()

DataFrame[summary: string, partition_date: string, order_id: string, commune: string, customer_id: string, employee_id: string, event_date: string, event_day: string, event_hour: string, event_minute: string, event_month: string, event_second: string, event_year: string, latitude: string, longitude: string, neighborhood: string, quantity_products: string, __index_level_0__: string]

In [6]:
# df_comunas.show()
!python


+--------------+--------------------+----------------+-----------+-----------+----------+---------+----------+------------+-----------+------------+----------+------------------+------------------+--------------------+-----------------+-----------------+
|partition_date|            order_id|         commune|customer_id|employee_id|event_date|event_day|event_hour|event_minute|event_month|event_second|event_year|          latitude|         longitude|        neighborhood|quantity_products|__index_level_0__|
+--------------+--------------------+----------------+-----------+-----------+----------+---------+----------+------------+-----------+------------+----------+------------------+------------------+--------------------+-----------------+-----------------+
|      03042024|10a44773-85b5-583...|        COMUNA 3|       4261|       1679|2024-04-03|       03|        23|          43|         04|          44|      2024|-75.54965000825186| 6.269627252440055|            MANRIQUE|                7

In [ ]:
# de botellas por comuna